In [1]:
from transformers import AutoTokenizer
from cruise.utilities.hdfs_io import hlist_files, hcopy
import os
import os.path as osp
import tempfile
from mariana.data.gpt.tokenization import CasterTokenizer
import pandas as pd
import numpy as np

tokenizer_type = 'bbpe' #'bbpe' 
data_type = 'hybrid'
if data_type == 'hybrid':
    data_path = 'hdfs://haruna/home/byte_data_aml_research/user/corpus_300B/sampled_v2.2.0/'
elif data_type == 'english':
    data_path = 'hdfs://haruna/home/byte_data_aml_research/user/corpus/sampled_v2.3.1/'

2023-02-23 16:55:02,859 WARNING 50131 [/mlx_devbox/users/yanshipeng/repo/182/mariana/mariana/data/gpt/tokenization.py:18]  Failed to import sentencepiece


In [2]:
if tokenizer_type == 'bbpe':
    # tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/tokenizer/fast_tokenizer_libcut_v2'
    # tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/tokenizer/gpt-neo'
    # tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/tokenizer/fast_tokenizer50k_en'
    # tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/tokenizer/fast_tokenizer_libcut_100k_code'
    # tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/tokenizer/fast_tokenizer_100k_v2'
    tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/qian.xian/fast_tokenizer_add_space_prefix_100k_code'
elif tokenizer_type == 'caster':
    tokenizer = 'hdfs://haruna/home/byte_data_aml_research/user/zhangzhi.joshua/tokenizer/zh_0620_newcut_caster_145665_lowercase'

tmp_dir = osp.join(tempfile.gettempdir(), osp.basename(tokenizer))
hcopy(tokenizer, tmp_dir)

if tokenizer_type == 'bbpe':
    tokenizer = AutoTokenizer.from_pretrained(tmp_dir)
elif tokenizer_type == 'caster':
    tokenizer = CasterTokenizer.from_pretrained(tmp_dir, max_len=-1)

copyToLocal: `/tmp/fast_tokenizer_add_space_prefix_100k_code/fast_tokenizer_add_space_prefix_100k_code/special_tokens_map.json': File exists
copyToLocal: `/tmp/fast_tokenizer_add_space_prefix_100k_code/fast_tokenizer_add_space_prefix_100k_code/tokenizer.json': File exists
copyToLocal: `/tmp/fast_tokenizer_add_space_prefix_100k_code/fast_tokenizer_add_space_prefix_100k_code/tokenizer_config.json': File exists


In [3]:
files = hlist_files([data_path])
import random
random.seed(0)
random.shuffle(files)
subfiles = files[:30]

In [4]:
from multiprocessing import Pool
def count_tokens(subfile):
    tmp_dir = tempfile.gettempdir()
    local_file = osp.join(tmp_dir, osp.basename(subfile))
    hcopy(subfile,  local_file)
    data = pd.read_parquet(local_file)
    tokens = [tokenizer.encode(x) for x in data['content_split']]
    parquet_tokens = np.array([len(tokens[i]) for i in range(len(tokens))]).sum()
    return parquet_tokens

In [ ]:
with Pool(processes=30) as p:
    results = p.map(count_tokens, subfiles)

copyToLocal: `/tmp/part-08292-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-13028-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-18435-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-10024-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-09692-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-02183-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-01734-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-14212-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-02189-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-07742-56bb4313-5e50-4293-a057-369db8d79bba-c000.gz.parquet': File exists
copyToLocal: `/tmp/part-10310-

In [ ]:
len(files)/30*sum(results) / 1e9

In [ ]:
247.76

In [ ]:
v2.2.0: 327.90B
pile: 200.37B